Some helpful imports. Installing `geopandas` can be a bit of a challenge sometimes.

In [ ]:
import networkx as nx
import random
import itertools
from collections import defaultdict
import cvxpy as cp
import copy
import numpy as np

import matplotlib.pyplot as plt

from scipy import stats

import geopandas as gpd
from datetime import datetime
import pandas as pd
import pickle
import copy
from IPython.display import clear_output
from tqdm import tqdm, tqdm_notebook
import re
import os, sys

import scipy.stats

import gurobipy

The following cell reads in the Cook County shapefile and somes some relabelling to get it into something which is marginally human-readable

In [ ]:


df = gpd.read_file("cook_il/shapefiles/cook_il_race_hisp.shp")

popcol = "total"
geolevels = ["state","county","subcounty","tractgroup","tract","blockgroup","block"]


cols = [
 'Id2',
 'Total_',
 'Population',
 'Populati_1',
 'Populati_2',
 'Populati_3',
 'Populati_4',
 'Populati_5',
 'Populati_6',
 'Two or Mor',
 'Two or M_1',
 'Two or M_2',
 'Two or M_3',
 'Two or M_4',
 'Two or M_5',
 'Two or M_6',
 'Two or M_7',
 'Two or M_8',
 'Two or M_9',
 'Two or M10',
 'Two or M11',
 'Two or M12',
 'Two or M13',
 'Two or M14',
 'Two or M15',
 'Two or M16',
 'Two or M17',
 'Two or M18',
 'Two or M19',
 'Two or M20',
 'Two or M21',
 'Two or M22',
 'Two or M23',
 'Two or M24',
 'Two or M25',
 'Two or M26',
 'Two or M27',
 'Two or M28',
 'Two or M29',
 'Two or M30',
 'Two or M31',
 'Two or M32',
 'Two or M33',
 'Two or M34',
 'Two or M35',
 'Two or M36',
 'Two or M37',
 'Two or M38',
 'Two or M39',
 'Two or M40',
 'Two or M41',
 'Two or M42',
 'Two or M43',
 'Two or M44',
 'Two or M45',
 'Two or M46',
 'Two or M47',
 'Two or M48',
 'Two or M49',
 'Two or M50',
 'Two or M51',
 'Two or M52',
 'Two or M53',
 'Two or M54',
 'Two or M55',
 'Two or M56',
 'Two or M57',
 'Two or M58',
 'Two or M59',
 'Two or M60',
 'Two or M61',
 'Two or M62',
 'Hispanic o',
 'Not Hispan',
 'Not Hisp_1',
 'Not Hisp_2',
 'Not Hisp_3',
 'Not Hisp_4',
 'Not Hisp_5',
 'Not Hisp_6',
 'Not Hisp_7',
 'Not Hisp_8',
 'Not Hisp_9',
 'Not Hisp10',
 'Not Hisp11',
 'Not Hisp12',
 'Not Hisp13',
 'Not Hisp14',
 'Not Hisp15',
 'Not Hisp16',
 'Not Hisp17',
 'Not Hisp18',
 'Not Hisp19',
 'Not Hisp20',
 'Not Hisp21',
 'Not Hisp22',
 'Not Hisp23',
 'Not Hisp24',
 'Not Hisp25',
 'Not Hisp26',
 'Not Hisp27',
 'Not Hisp28',
 'Not Hisp29',
 'Not Hisp30',
 'Not Hisp31',
 'Not Hisp32',
 'Not Hisp33',
 'Not Hisp34',
 'Not Hisp35',
 'Not Hisp36',
 'Not Hisp37',
 'Not Hisp38',
 'Not Hisp39',
 'Not Hisp40',
 'Not Hisp41',
 'Not Hisp42',
 'Not Hisp43',
 'Not Hisp44',
 'Not Hisp45',
 'Not Hisp46',
 'Not Hisp47',
 'Not Hisp48',
 'Not Hisp49',
 'Not Hisp50',
 'Not Hisp51',
 'Not Hisp52',
 'Not Hisp53',
 'Not Hisp54',
 'Not Hisp55',
 'Not Hisp56',
 'Not Hisp57',
 'Not Hisp58',
 'Not Hisp59',
 'Not Hisp60',
 'Not Hisp61',
 'Not Hisp62',
 'Not Hisp63',
 'Not Hisp64',
 'Not Hisp65',
 'Not Hisp66',
 'Not Hisp67',
 'Not Hisp68',
 'Not Hisp69',
 'Not Hisp70',
 'geometry']




new_col_names = [
'id',
'total',
'r_1',
'r_1-w',
'r_1-b',
'r_1-amin',
'r_1-a',
'r_1-nhpi',
'r_1-other',
'r_2_plus',
'r_2',
'r_2-w-b',
'r_2-w-amin',
'r_2-w-a',
'r_2-w-nhpi',
'r_2-w-other',
'r_2-b-amin',
'r_2-b-a',
'r_2-b-nhpi',
'r_2-b-other',
'r_2-amin-a',
'r_2-amin-nhpi',
'r_2-amin-other',
'r_2-a-nhpi',
'r_2-a-other',
'r_2-nhpi-other',
'r_3',
'r_3-w-b-amin',
'r_3-w-b-a',
'r_3-w-b-nhpi',
'r_3-w-b-other',
'r_3-w-amin-a',
'r_3-w-amin-nhpi',
'r_3-w-amin-other',
'r_3-w-a-nhpi',
'r_3-w-a-other',
'r_3-w-nhpi-other',
'r_3-b-amin-a',
'r_3-b-amin-nhpi',
'r_3-b-amin-other',
'r_3-b-a-nhpi',
'r_3-b-a-other',
'r_3-b-nhpi-other',
'r_3-amin-a-nhpi',
'r_3-amin-a-other',
'r_3-amin-nhpi-other',
'r_3-a-nhpi-other',
'r_4',
'r_4-w-b-amin-a',
'r_4-w-b-amin-nhpi',
'r_4-w-b-amin-other',
'r_4-w-b-a-nhpi',
'r_4-w-b-a-other',
'r_4-w-b-nhpi-other',
'r_4-w-amin-a-nhpi',
'r_4-w-amin-a-other',
'r_4-w-amin-nhpi-other',
'r_4-w-a-nhpi-other',
'r_4-b-amin-a-nhpi',
'r_4-b-amin-a-other',
'r_4-b-amin-nhpi-other',
'r_4-b-a-nhpi-other',
'r_4-amin-a-nhpi-other',
'r_5',
'r_5-w-b-amin-a-nhpi',
'r_5-w-b-amin-a-other',
'r_5-w-b-amin-nhpi-other',
'r_5-w-b-a-nhpi-other',
'r_5-w-amin-a-nhpi-other',
'r_5-b-amin-a-nhpi-other',
'r_6',
'r_6-w-b-amin-a-nhpi-other',
'hisp',
'nohisp',
'nohisp_r_1',
'nohisp_r_1-w',
'nohisp_r_1-b',
'nohisp_r_1-amin',
'nohisp_r_1-a',
'nohisp_r_1-nhpi',
'nohisp_r_1-other',
'nohisp_r_2_plus',
'nohisp_r_2',
'nohisp_r_2-w-b',
'nohisp_r_2-w-amin',
'nohisp_r_2-w-a',
'nohisp_r_2-w-nhpi',
'nohisp_r_2-w-other',
'nohisp_r_2-b-amin',
'nohisp_r_2-b-a',
'nohisp_r_2-b-nhpi',
'nohisp_r_2-b-other',
'nohisp_r_2-amin-a',
'nohisp_r_2-amin-nhpi',
'nohisp_r_2-amin-other',
'nohisp_r_2-a-nhpi',
'nohisp_r_2-a-other',
'nohisp_r_2-nhpi-other',
'nohisp_r_3',
'nohisp_r_3-w-b-amin',
'nohisp_r_3-w-b-a',
'nohisp_r_3-w-b-nhpi',
'nohisp_r_3-w-b-other',
'nohisp_r_3-w-amin-a',
'nohisp_r_3-w-amin-nhpi',
'nohisp_r_3-w-amin-other',
'nohisp_r_3-w-a-nhpi',
'nohisp_r_3-w-a-other',
'nohisp_r_3-w-nhpi-other',
'nohisp_r_3-b-amin-a',
'nohisp_r_3-b-amin-nhpi',
'nohisp_r_3-b-amin-other',
'nohisp_r_3-b-a-nhpi',
'nohisp_r_3-b-a-other',
'nohisp_r_3-b-nhpi-other',
'nohisp_r_3-amin-a-nhpi',
'nohisp_r_3-amin-a-other',
'nohisp_r_3-amin-nhpi-other',
'nohisp_r_3-a-nhpi-other',
'nohisp_r_4',
'nohisp_r_4-w-b-amin-a',
'nohisp_r_4-w-b-amin-nhpi',
'nohisp_r_4-w-b-amin-other',
'nohisp_r_4-w-b-a-nhpi',
'nohisp_r_4-w-b-a-other',
'nohisp_r_4-w-b-nhpi-other',
'nohisp_r_4-w-amin-a-nhpi',
'nohisp_r_4-w-amin-a-other',
'nohisp_r_4-w-amin-nhpi-other',
'nohisp_r_4-w-a-nhpi,-other',
'nohisp_r_4-b-amin-a-nhpi',
'nohisp_r_4-b-amin-a-other',
'nohisp_r_4-b-amin-nhpi-other',
'nohisp_r_4-b-a-nhpi-other',
'nohisp_r_4-amin-a-nhpi-other',
'nohisp_r_5',
'nohisp_r_5-w-b-amin-a-nhpi',
'nohisp_r_5-w-b-amin-a-other',
'nohisp_r_5-w-b-amin-nhpi-other',
'nohisp_r_5-w-b-a-nhpi-other',
'nohisp_r_5-w-amin-a-nhpi-other',
'nohisp_r_5-b-amin-a-nhpi-other',
'nohisp_r_6',
'nohisp_r_6-w-b-amin-a-nhpi-other',
'geometry'
]





cols_threeplus = [
    'nohisp_r_3',
'nohisp_r_3-w-b-amin',
'nohisp_r_3-w-b-a',
'nohisp_r_3-w-b-nhpi',
'nohisp_r_3-w-b-other',
'nohisp_r_3-w-amin-a',
'nohisp_r_3-w-amin-nhpi',
'nohisp_r_3-w-amin-other',
'nohisp_r_3-w-a-nhpi',
'nohisp_r_3-w-a-other',
'nohisp_r_3-w-nhpi-other',
'nohisp_r_3-b-amin-a',
'nohisp_r_3-b-amin-nhpi',
'nohisp_r_3-b-amin-other',
'nohisp_r_3-b-a-nhpi',
'nohisp_r_3-b-a-other',
'nohisp_r_3-b-nhpi-other',
'nohisp_r_3-amin-a-nhpi',
'nohisp_r_3-amin-a-other',
'nohisp_r_3-amin-nhpi-other',
'nohisp_r_3-a-nhpi-other',
'nohisp_r_4',
'nohisp_r_4-w-b-amin-a',
'nohisp_r_4-w-b-amin-nhpi',
'nohisp_r_4-w-b-amin-other',
'nohisp_r_4-w-b-a-nhpi',
'nohisp_r_4-w-b-a-other',
'nohisp_r_4-w-b-nhpi-other',
'nohisp_r_4-w-amin-a-nhpi',
'nohisp_r_4-w-amin-a-other',
'nohisp_r_4-w-amin-nhpi-other',
'nohisp_r_4-w-a-nhpi,-other',
'nohisp_r_4-b-amin-a-nhpi',
'nohisp_r_4-b-amin-a-other',
'nohisp_r_4-b-amin-nhpi-other',
'nohisp_r_4-b-a-nhpi-other',
'nohisp_r_4-amin-a-nhpi-other',
'nohisp_r_5',
'nohisp_r_5-w-b-amin-a-nhpi',
'nohisp_r_5-w-b-amin-a-other',
'nohisp_r_5-w-b-amin-nhpi-other',
'nohisp_r_5-w-b-a-nhpi-other',
'nohisp_r_5-w-amin-a-nhpi-other',
'nohisp_r_5-b-amin-a-nhpi-other',
'nohisp_r_6',
'nohisp_r_6-w-b-amin-a-nhpi-other',
    
'r_3',
'r_3-w-b-amin',
'r_3-w-b-a',
'r_3-w-b-nhpi',
'r_3-w-b-other',
'r_3-w-amin-a',
'r_3-w-amin-nhpi',
'r_3-w-amin-other',
'r_3-w-a-nhpi',
'r_3-w-a-other',
'r_3-w-nhpi-other',
'r_3-b-amin-a',
'r_3-b-amin-nhpi',
'r_3-b-amin-other',
'r_3-b-a-nhpi',
'r_3-b-a-other',
'r_3-b-nhpi-other',
'r_3-amin-a-nhpi',
'r_3-amin-a-other',
'r_3-amin-nhpi-other',
'r_3-a-nhpi-other',
'r_4',
'r_4-w-b-amin-a',
'r_4-w-b-amin-nhpi',
'r_4-w-b-amin-other',
'r_4-w-b-a-nhpi',
'r_4-w-b-a-other',
'r_4-w-b-nhpi-other',
'r_4-w-amin-a-nhpi',
'r_4-w-amin-a-other',
'r_4-w-amin-nhpi-other',
'r_4-w-a-nhpi-other',
'r_4-b-amin-a-nhpi',
'r_4-b-amin-a-other',
'r_4-b-amin-nhpi-other',
'r_4-b-a-nhpi-other',
'r_4-amin-a-nhpi-other',
'r_5',
'r_5-w-b-amin-a-nhpi',
'r_5-w-b-amin-a-other',
'r_5-w-b-amin-nhpi-other',
'r_5-w-b-a-nhpi-other',
'r_5-w-amin-a-nhpi-other',
'r_5-b-amin-a-nhpi-other',
'r_6',
'r_6-w-b-amin-a-nhpi-other',    
    
]





noisy_cols = [
'total',
'r_1',
'r_1-w',
'r_1-b',
'r_1-amin',
'r_1-a',
'r_1-nhpi',
'r_1-other',
'r_2_plus',
'r_2',
'r_2-w-b',
'r_2-w-amin',
'r_2-w-a',
'r_2-w-nhpi',
'r_2-w-other',
'r_2-b-amin',
'r_2-b-a',
'r_2-b-nhpi',
'r_2-b-other',
'r_2-amin-a',
'r_2-amin-nhpi',
'r_2-amin-other',
'r_2-a-nhpi',
'r_2-a-other',
'r_2-nhpi-other',
'r_3_plus',
'hisp',
'nohisp',
'nohisp_r_1',
'nohisp_r_1-w',
'nohisp_r_1-b',
'nohisp_r_1-amin',
'nohisp_r_1-a',
'nohisp_r_1-nhpi',
'nohisp_r_1-other',
'nohisp_r_2_plus',
'nohisp_r_2',
'nohisp_r_2-w-b',
'nohisp_r_2-w-amin',
'nohisp_r_2-w-a',
'nohisp_r_2-w-nhpi',
'nohisp_r_2-w-other',
'nohisp_r_2-b-amin',
'nohisp_r_2-b-a',
'nohisp_r_2-b-nhpi',
'nohisp_r_2-b-other',
'nohisp_r_2-amin-a',
'nohisp_r_2-amin-nhpi',
'nohisp_r_2-amin-other',
'nohisp_r_2-a-nhpi',
'nohisp_r_2-a-other',
'nohisp_r_2-nhpi-other',
'nohisp_r_3_plus'
]


check_cols = [
'r_1',
'r_1-w',
'r_1-b',
'r_1-amin',
'r_1-a',
'r_1-nhpi',
'r_1-other',
'r_2_plus',
'r_2',
'r_2-w-b',
'r_2-w-amin',
'r_2-w-a',
'r_2-w-nhpi',
'r_2-w-other',
'r_2-b-amin',
'r_2-b-a',
'r_2-b-nhpi',
'r_2-b-other',
'r_2-amin-a',
'r_2-amin-nhpi',
'r_2-amin-other',
'r_2-a-nhpi',
'r_2-a-other',
'r_2-nhpi-other',
'r_3_plus',
]





vert_cols = [
    
    
'r_1-w',
'r_1-b',
'r_1-amin',
'r_1-a',
'r_1-nhpi',
'r_1-other',

'r_2-w-b',
'r_2-w-amin',
'r_2-w-a',
'r_2-w-nhpi',
'r_2-w-other',
'r_2-b-amin',
'r_2-b-a',
'r_2-b-nhpi',
'r_2-b-other',
'r_2-amin-a',
'r_2-amin-nhpi',
'r_2-amin-other',
'r_2-a-nhpi',
'r_2-a-other',
'r_2-nhpi-other',
'r_3_plus',

'nohisp_r_1-w',
'nohisp_r_1-b',
'nohisp_r_1-amin',
'nohisp_r_1-a',
'nohisp_r_1-nhpi',
'nohisp_r_1-other',
'nohisp_r_2-w-b',
'nohisp_r_2-w-amin',
'nohisp_r_2-w-a',
'nohisp_r_2-w-nhpi',
'nohisp_r_2-w-other',
'nohisp_r_2-b-amin',
'nohisp_r_2-b-a',
'nohisp_r_2-b-nhpi',
'nohisp_r_2-b-other',
'nohisp_r_2-amin-a',
'nohisp_r_2-amin-nhpi',
'nohisp_r_2-amin-other',
'nohisp_r_2-a-nhpi',
'nohisp_r_2-a-other',
'nohisp_r_2-nhpi-other',
'nohisp_r_3_plus'
    
    
    
]



df = df.rename(columns={  cols[i] : new_col_names[i] for i in range(len(cols))    })


df = df[new_col_names]


df = df.fillna(0)

for c in new_col_names[1:-1]:
    df["new"] = df[c].str.replace('\(([^)]+)\)',"",regex=True)
    
    df[c] = df["new"].astype("float")



## collapse all r_3++ into new r_3_plus
df['r_3_plus'] = (df['r_3'] + df['r_4'] + df['r_5'] + df["r_6"]).astype(int)

df['nohisp_r_3_plus'] = (df['nohisp_r_3'] + df['nohisp_r_4'] + df['nohisp_r_5'] + df['nohisp_r_6'])


## drop extra r_3,4,5,6 cols
df = df.drop(columns = cols_threeplus)

df = df[df["total"] != 0]



# add geographic hierarchy columns:


df["state"]      = df["id"].str[:2]
df["county"]     = df["id"].str[:5]
df["subcounty"]  = df["id"].str[:7]
df["tractgroup"] = df["id"].str[:9]
df["tract"]      = df["id"].str[:11]
df["blockgroup"] = df["id"].str[:12]
df["block"]      = df["id"].str[:]

This cell does some quick sanity checks of the data, making sure that the constraints we are about to define are satisfied by the raw data.

In [ ]:
# data checks: should all of these should be 1


## check that everyone is 1, or 2+ races
print(np.prod(df["total"] == df["r_1"] + df["r_2_plus"]))
print(np.prod(df["nohisp"] == df["nohisp_r_1"] + df["nohisp_r_2_plus"]))




## check that everyone is 1, 2, or 3+ races
print(np.prod(df["total"] == df["r_1"] + df["r_2"] + df["r_3_plus"]))
print(np.prod(df["nohisp"] == df["nohisp_r_1"] + df["nohisp_r_2"] + df["nohisp_r_3_plus"]))


## check that "2+" equals 2 and 3+
print(np.prod(df["r_2_plus"] ==  df["r_2"] + df["r_3_plus"]))
print(np.prod(df["nohisp_r_2_plus"] ==  df["nohisp_r_2"] + df["nohisp_r_3_plus"]))

## check that everyone is either hisp/lat or not
print(np.prod(df["total"] == df["nohisp"] + df["hisp"]))


## check that all the single-race people are accounted for
print(np.prod( df["r_1"] == df['r_1-w'] + df['r_1-b'] + df['r_1-amin'] + df['r_1-a'] + df['r_1-nhpi'] + df['r_1-other']  ))
print(np.prod( df["nohisp_r_1"] == df['nohisp_r_1-w'] + df['nohisp_r_1-b'] + df['nohisp_r_1-amin'] + df['nohisp_r_1-a'] + df['nohisp_r_1-nhpi'] + df['nohisp_r_1-other']  ))


## check that all the two race people are accounted for
print(np.prod(df['r_2'] == df['r_2-w-b'] + df['r_2-w-amin'] + df['r_2-w-a'] + df['r_2-w-nhpi'] + df['r_2-w-other'] + df['r_2-b-amin'] + df['r_2-b-a'] + df['r_2-b-nhpi'] + df['r_2-b-other'] + df['r_2-amin-a'] + df['r_2-amin-nhpi'] + df['r_2-amin-other'] + df['r_2-a-nhpi'] + df['r_2-a-other'] + df['r_2-nhpi-other']))
print(np.prod(df['nohisp_r_2'] == df['nohisp_r_2-w-b'] + df['nohisp_r_2-w-amin'] + df['nohisp_r_2-w-a'] + df['nohisp_r_2-w-nhpi'] + df['nohisp_r_2-w-other'] + df['nohisp_r_2-b-amin'] + df['nohisp_r_2-b-a'] + df['nohisp_r_2-b-nhpi'] + df['nohisp_r_2-b-other'] + df['nohisp_r_2-amin-a'] + df['nohisp_r_2-amin-nhpi'] + df['nohisp_r_2-amin-other'] + df['nohisp_r_2-a-nhpi'] + df['nohisp_r_2-a-other'] + df['nohisp_r_2-nhpi-other']))



# check that the nohisp values are less than total

for c in check_cols:
    print(np.prod(df[c]>= df["nohisp_"+c]))



Here we define the relevant constraints for within a unit.  Things like "hispanic population plus non-hispanic population is equal to the total population" go here.

In [ ]:
# the vertical constraints (sum over all subunits equals unit) will be handled automatically
# these are the horizontal constraints 
# all of these will be linear
# form is ( [list of attrs] , relation, [list of attrs]   )
# i.e. ( ['_one','_two','_threeplus'], '=' , ['pop']) 


## invariants

invariants = [


## check that everyone is 1, or 2+ races
(["total"], '=', ["r_1","r_2_plus"]),
(["nohisp"] ,'=', ["nohisp_r_1","nohisp_r_2_plus"]),




## check that everyone is 1, 2, or 3+ races
(["total"], '=', ["r_1","r_2","r_3_plus"]),
(["nohisp"] ,'=', ["nohisp_r_1","nohisp_r_2","nohisp_r_3_plus"]),



## check that "2+" equals 2 and 3+
(["r_2_plus"], '=', ["r_2","r_3_plus"]),
(["nohisp_r_2_plus"], '=', ["nohisp_r_2","nohisp_r_3_plus"]),


## check that everyone is either hisp/lat or not
(["total"], '=', ["nohisp","hisp"]),


## check that all the single-race people are accounted for
(["r_1"], "=",  ['r_1-w','r_1-b','r_1-amin','r_1-a','r_1-nhpi','r_1-other']  ),
(["nohisp_r_1"] , "=", ['nohisp_r_1-w','nohisp_r_1-b','nohisp_r_1-amin','nohisp_r_1-a','nohisp_r_1-nhpi','nohisp_r_1-other']  ),


## check that all the two race people are accounted for
(['r_2'] ,"=", ['r_2-w-b','r_2-w-amin','r_2-w-a','r_2-w-nhpi','r_2-w-other','r_2-b-amin','r_2-b-a','r_2-b-nhpi','r_2-b-other','r_2-amin-a','r_2-amin-nhpi','r_2-amin-other','r_2-a-nhpi','r_2-a-other','r_2-nhpi-other']),
(['nohisp_r_2'] ,"=", ['nohisp_r_2-w-b','nohisp_r_2-w-amin','nohisp_r_2-w-a','nohisp_r_2-w-nhpi','nohisp_r_2-w-other','nohisp_r_2-b-amin','nohisp_r_2-b-a','nohisp_r_2-b-nhpi','nohisp_r_2-b-other','nohisp_r_2-amin-a','nohisp_r_2-amin-nhpi','nohisp_r_2-amin-other','nohisp_r_2-a-nhpi','nohisp_r_2-a-other','nohisp_r_2-nhpi-other'])



# check that the nohisp values are less than total
]
for c in check_cols:
    invariants.append((["nohisp_{}".format(c)], "<=", [c]))










Some helper functions to generate random noise.

In [ ]:
sensitivity = 2


def unif_rand(n):
    return random.randint(-n,n)

def geom_noise(p):
    if p == 0: return 0
    x =  np.random.geometric(p) - np.random.geometric(p)
    return x


def add_noise(df, gen, param):
    df_noisy = df.copy()
    for c in noisy_cols:
        noise = [ gen(param) for i in range(len(df))  ]
        df_noisy[c] += noise
        df_noisy[c] = df_noisy[c].clip(lower=0)
    return df_noisy



It is going to be useful to precompute the number of blocks constituting each unit.  The `block_counts` dictionary holds that information

In [ ]:
block_counts = defaultdict(int)

for level in geolevels[:-1]:
    print(level, len(set(list(df[level]))))
    count = 0
    for name in set(list(df[level])):
        count +=1
        #print(count, end=', ')
        block_counts[(level,name)] = df[level].value_counts()[name]
    #clear_output()   
    
def num_blocks(level, name):
    #return nc[level].value_counts()[name]
    
    if level == "block": return 1
    return block_counts[(level,name)]

The L_2 optimization problem.

In [ ]:
def solve_l2(df, df_noisy, level, ischild=True, verbose=False):
    df_opt = df_noisy.copy()
#     print("called l2",level)
#     print(df["pop"])
#     print(df_noisy["pop"])
    
    
    objective = 0
    constraints = []
    for c in noisy_cols:
        df_opt[c] = list(cp.Variable(len(df_opt[c])))

    
    for inv in invariants:
        #print(inv, end=' ')
        if inv[1] == '=':
            #print(list(df_opt[inv[0]].sum(axis=1)))
           
            lhs = list(df_opt[inv[0]].sum(axis=1))
            rhs = list(df_opt[inv[2]].sum(axis=1))
            for l,r in zip(lhs,rhs):
                constraints.append((l-r==0))
                
        if inv[1] == '<=':
            lhs = list(df_opt[inv[0]].sum(axis=1))
            rhs = list(df_opt[inv[2]].sum(axis=1))

            for l,r in zip(lhs,rhs):
                constraints.append((l-r<=0))
                
    levelids = list(df_noisy[level])
    for c in noisy_cols:
        #print(c , end = ' ')
        if c in vert_cols: constraints.append(  (sum(df_opt[c]) == sum(df[c])    )      )
        for n,o,l in zip(list(df_noisy[c]),list(df_opt[c]),levelids):
            
            
            objective += ((n-o)**2)
            if c != popcol:
                
                constraints.append((o>=0))
            else:
                poptar = num_blocks(level, l     )
                constraints.append((o>= float(poptar)))
                
    #print("CONSTRUCTED")
    prob = cp.Problem(cp.Minimize(objective),constraints)
    #print("INITIALIZED")
    prob.solve( solver="GUROBI", verbose=verbose)
    
    
    for c in noisy_cols:
        df_opt[c] = [ np.around(e.value.item(), decimals=5) for e in list(df_opt[c])        ]



    return df_opt



In [ ]:
The L_1 optimization problem.

In [ ]:
def solve_l1(df, df_noisy, level, ischild=True, verbose=False):
    

    df_opt = df_noisy.copy()
#     print("called l1", level)
#     print(df["pop"])
#     print(df_noisy["pop"])


    
    objective = 0
    constraints = []
    
    for c in noisy_cols:
        df_opt[c] = list(cp.Variable(len(df_opt[c]), integer=True))

    
    for inv in invariants:
        #print(inv, end=' ')

        if inv[1] == '=':
            #print(list(df_opt[inv[0]].sum(axis=1)))
            
            lhs = list(df_opt[inv[0]].sum(axis=1))
            rhs = list(df_opt[inv[2]].sum(axis=1))

            for l,r in zip(lhs,rhs):
                constraints.append((l-r==0))
                
        elif inv[1] == '<=':
            lhs = list(df_opt[inv[0]].sum(axis=1))
            rhs = list(df_opt[inv[2]].sum(axis=1))

            for l,r in zip(lhs,rhs):
                constraints.append((l-r<=0))                
                
                
                
    levelids = list(df_noisy[level])
    for c in noisy_cols:
        #print(c , end = ' ')

        if c in vert_cols: constraints.append(  (sum(df_opt[c]) == sum(df[c])    )      )
        for n,o,l in zip(list(df_noisy[c]),list(df_opt[c]),levelids):
            
            
            objective +=  (( o - np.floor(n) ) * 
                                 ( n - np.floor(n) ))
            constraints.append( (cp.abs( o - n   ) <= 1 )  )

            if c != popcol:
                constraints.append((o>=0))
            else:
                poptar = num_blocks(level, l     )
                constraints.append((o>= float(poptar)))
    
    
    
    
    #print("CONSTRUCTED")
    prob = cp.Problem(cp.Minimize(objective),constraints)
    #print("INITIALIZED")
    prob.solve( solver="GUROBI", verbose=verbose)
    
    
    for c in noisy_cols:
        df_opt[c] = [ np.rint(e.value.item()) for e in list(df_opt[c])        ]


#     print("done in {}".format(datetime.now()-tic))

    return df_opt    

A helper function to gather the child dataframes for a given parent unit.

In [ ]:
def get_subunit_dfs(df, parent_field, child_field):
    d = {}
    parents = set(df[parent_field])
    
    for unit in parents:
        df_sub = df.loc[df[parent_field]==unit,:]
        d[unit] = (df_sub.groupby(child_field).sum().reset_index() )
       
    
    
    return d#{ unit :df.loc[df[parent_field]==unit,:].groupby(child_field).sum().reset_index() for unit in set(df[parent_field]) } 

In [ ]:
def sort_num_blocks(keys,level):
    l = [ block_counts[(k,level)] for k in keys   ]
    
    return [x for _,x in sorted(zip(l,keys))]

The main loop to generate a privatized data frame.  Unless the amount of noise being added is very very large, the optimization problems shouldn't fail to solve.  If the solver fails 50 times in a row on a particular unit (fresh randomness is drawn each time), the whole algorithm will abort.  In generating the data for the paper, we never failed more than twice on any unit.

In [ ]:
failure = None

def make_private(df, levels, noise_func, noise_params,verbose):
    print(noise_params,levels)
    global failure
    failure = None
    print("Starting")
   
    _df_tmp = df.groupby(levels[0]).sum().reset_index()
    _df_tmp_noisy = add_noise(_df_tmp, noise_func, noise_params[0])
    _df_tmp_opt = solve_l2(_df_tmp, _df_tmp_noisy, levels[0], ischild=False, verbose=verbose)
    _dfs_tmp_opt = {"master": solve_l1(_df_tmp, _df_tmp_opt, levels[0], ischild=False, verbose=verbose)}
   
    print("done {} (top level)".format(levels[0]))
   
    for l in range(1,len(levels)):
        print("starting {}s".format(levels[l]))
        _dfs_dict = get_subunit_dfs(df, levels[l-1], levels[l])
        _dfs_dict_noisy = {k:add_noise(v,noise_func, noise_params[l]) for k,v in _dfs_dict.items()}
        _dfs_dict_opt = {}
        for k in tqdm_notebook(sort_num_blocks(list(_dfs_dict.keys()), levels[l-1])):
            #print("doing {}s in {}".format(levels[l], k))
       
       
            for v in _dfs_tmp_opt.values():
                df_sub = v.loc[v[levels[l-1]] == k,:]
                if df_sub.shape[0] != 0:
                    _f = False
                    _failcount = 0
                    while not _f and _failcount < 50:
                        try:
                            _t = solve_l2(df_sub, _dfs_dict_noisy[k],levels[l], verbose=verbose)
                            _dfs_dict_opt[k] = solve_l1(df_sub,_t, levels[l], verbose=verbose)
                            _f = True
                        except:
                            failure = (df_sub, _dfs_dict_noisy[k],levels[l] )
                            _dfs_dict_noisy[k] = add_noise(_dfs_dict[k],noise_func,noise_params[l])
                            print("FAILED ON {} {} {}".format(levels[l], k, _failcount))
                            _failcount +=1
                    if _failcount > 50: return None
                   
        _dfs_tmp_opt = copy.deepcopy(_dfs_dict_opt)
   
   
    priv_dfs = [v for v in _dfs_tmp_opt.values()]

    df_private = priv_dfs[0]
    for priv_df in priv_dfs[1:]:
        df_private = pd.concat([df_private, priv_df.reset_index(drop=True)], axis=0)
   
    return df_private.sort_values(levels[-1])

This makes a differentially private copy of the dataframe as used in the paper.

In [ ]:
private = make_private(df,geolevels[1:-1], geom_noise, [.5]*5,verbose=0)
